# Concurrent Multi-Agent Workflow with Visualization

This notebook demonstrates a **fan-out/fan-in pattern** with multiple AI agents working concurrently, combined with **workflow visualization** capabilities using Mermaid diagrams, DiGraph, and SVG export.

## Key Concepts

### Fan-Out/Fan-In Pattern
- **Fan-Out**: Dispatch the same prompt to multiple domain expert agents simultaneously
- **Parallel Processing**: All agents process independently and concurrently
- **Fan-In**: Aggregate responses from all agents into a consolidated output
- **Domain Expertise**: Each agent specializes in a specific area (research, marketing, legal)

### Multi-Agent Orchestration
- **AgentExecutor**: Wraps Azure OpenAI agents with workflow integration
- **Target Routing**: `target_id` directs requests to specific agents
- **Response Aggregation**: Collect and organize outputs by agent ID
- **Structured Output**: Dataclass for organized multi-agent insights

### Workflow Visualization
- **WorkflowViz**: Generate visual representations of workflow graphs
- **Mermaid Diagrams**: Text-based flowcharts for documentation
- **DiGraph**: NetworkX graph objects for programmatic analysis
- **SVG Export**: High-quality vector graphics (requires GraphViz)

## Use Case: Product Launch Analysis

Analyze a product launch idea from three expert perspectives:

```
                    Dispatcher
                   /     |     \
            Researcher Marketer Legal
                   \     |     /
                    Aggregator
```

**Experts:**
- 🔬 **Researcher**: Market analysis, opportunities, and risks
- 📈 **Marketer**: Value propositions and target messaging
- ⚖️ **Legal**: Compliance, disclaimers, and policy concerns

## What This Example Shows

1. **Custom Executors**: Build dispatcher and aggregator with `@handler` decorator
2. **Multi-Agent Coordination**: Orchestrate three AI agents in parallel
3. **Structured Aggregation**: Organize responses into `AggregatedInsights` dataclass
4. **Workflow Visualization**: Generate Mermaid, DiGraph, and SVG representations
5. **Event Streaming**: Monitor agent execution with `AgentRunEvent`
6. **Azure OpenAI Integration**: Enterprise-ready AI agent configuration

## Setup

Import required modules and configure Azure OpenAI.

### Prerequisites:
- Azure OpenAI deployment with GPT model
- Azure CLI authentication: Run `az login` before executing
- For SVG export: `pip install agent-framework[viz] --pre` and install GraphViz binaries
- Environment variables for Azure OpenAI endpoint and deployment

In [ ]:
import asyncio
from dataclasses import dataclass

import os
from dotenv import load_dotenv
from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    AgentRunEvent,
    ChatMessage,
    Executor,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    WorkflowViz,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from typing_extensions import Never

print("✅ Imports complete")
print("⚠️  Make sure you've run 'az login' for Azure authentication")
print("💡 For SVG export, install: pip install agent-framework[viz] --pre")
# Load environment variables from .env file
load_dotenv('../../.env')


## Define the Dispatcher Executor

The **DispatchToExperts** executor fans out the prompt to all domain expert agents.

### Key Features:
- **Fan-Out Logic**: Sends same prompt to multiple agents via `target_id`
- **Expert IDs**: Maintains list of agent IDs to dispatch to
- **Message Wrapping**: Converts prompt string into `AgentExecutorRequest`
- **Parallel Dispatch**: All agents receive requests simultaneously

### Constructor:
```python
__init__(self, expert_ids: list[str], id: str | None = None)
```
- **expert_ids**: List of agent executor IDs (e.g., ["researcher", "marketer", "legal"])
- **id**: Optional custom executor ID (defaults to "dispatch_to_experts")

### Handler Method:
```python
@handler
async def dispatch(self, prompt: str, ctx: WorkflowContext[AgentExecutorRequest])
```
- **@handler**: Registers method as the executor's main logic
- **prompt**: Input string to analyze from multiple perspectives
- **ctx.send_message()**: Sends request to each agent with `target_id`

### Dispatch Flow:
1. Wrap prompt as `ChatMessage` with `Role.USER`
2. Create `AgentExecutorRequest` with `should_respond=True`
3. Send to each expert agent using `target_id` for routing
4. Framework handles parallel execution automatically

In [ ]:
class DispatchToExperts(Executor):
    """Dispatches the incoming prompt to all expert agent executors (fan-out)."""

    def __init__(self, expert_ids: list[str], id: str | None = None):
        super().__init__(id=id or "dispatch_to_experts")
        self._expert_ids = expert_ids

    @handler
    async def dispatch(self, prompt: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
        # Wrap the incoming prompt as a user message for each expert and request a response.
        initial_message = ChatMessage(Role.USER, text=prompt)
        
        print(f"\n📤 Dispatching prompt to {len(self._expert_ids)} experts:")
        print(f"   Prompt: {prompt[:100]}..." if len(prompt) > 100 else f"   Prompt: {prompt}")
        print(f"   Experts: {', '.join(self._expert_ids)}\n")
        
        for expert_id in self._expert_ids:
            await ctx.send_message(
                AgentExecutorRequest(messages=[initial_message], should_respond=True),
                target_id=expert_id,
            )


print("✅ DispatchToExperts executor defined")

## Define the Aggregated Insights Data Model

The **AggregatedInsights** dataclass structures the consolidated output from all agents.

### Fields:
- **research**: Research findings (market analysis, opportunities, risks)
- **marketing**: Marketing strategy (value propositions, messaging)
- **legal**: Legal/compliance notes (constraints, disclaimers, policies)

### Benefits:
- ✅ **Type Safety**: Structured data with clear field names
- ✅ **Documentation**: Self-documenting output structure
- ✅ **Validation**: Type checking ensures all fields are present
- ✅ **Extensibility**: Easy to add more expert perspectives

In [ ]:
@dataclass
class AggregatedInsights:
    """Structured output from the aggregator."""
    research: str
    marketing: str
    legal: str


print("✅ AggregatedInsights dataclass defined")
print(f"   Fields: {list(AggregatedInsights.__annotations__.keys())}")

## Define the Aggregator Executor

The **AggregateInsights** executor fans in responses from all agents and consolidates them.

### Key Features:
- **Fan-In Logic**: Receives list of `AgentExecutorResponse` from all agents
- **Response Mapping**: Organizes responses by `executor_id`
- **Structured Aggregation**: Creates `AggregatedInsights` dataclass
- **Formatted Output**: Generates readable consolidated report

### Constructor:
```python
__init__(self, expert_ids: list[str], id: str | None = None)
```
- **expert_ids**: Expected agent IDs for response mapping
- **id**: Optional custom executor ID (defaults to "aggregate_insights")

### Handler Method:
```python
@handler
async def aggregate(self, results: list[AgentExecutorResponse], ctx: WorkflowContext[Never, str])
```
- **results**: List of agent responses (one per expert)
- **Context Type**: `WorkflowContext[Never, str]` - Never returns, yields string output

### Aggregation Flow:
1. Map responses to text by `executor_id` (researcher, marketer, legal)
2. Extract text from each `AgentExecutorResponse.agent_run_response.text`
3. Create `AggregatedInsights` dataclass with organized responses
4. Format consolidated report with sections for each expert
5. Yield final output to complete workflow

### Output Format:
```
Consolidated Insights
====================

Research Findings:
[research analysis]

Marketing Angle:
[marketing strategy]

Legal/Compliance Notes:
[legal concerns]
```

In [ ]:
class AggregateInsights(Executor):
    """Aggregates expert agent responses into a single consolidated result (fan-in)."""

    def __init__(self, expert_ids: list[str], id: str | None = None):
        super().__init__(id=id or "aggregate_insights")
        self._expert_ids = expert_ids

    @handler
    async def aggregate(self, results: list[AgentExecutorResponse], ctx: WorkflowContext[Never, str]) -> None:
        print(f"\n📥 Aggregating responses from {len(results)} experts...\n")
        
        # Map responses to text by executor id for a simple, predictable demo.
        by_id: dict[str, str] = {}
        for r in results:
            # AgentExecutorResponse.agent_run_response.text contains concatenated assistant text
            by_id[r.executor_id] = r.agent_run_response.text
            print(f"   ✅ Received response from: {r.executor_id}")

        research_text = by_id.get("researcher", "")
        marketing_text = by_id.get("marketer", "")
        legal_text = by_id.get("legal", "")

        aggregated = AggregatedInsights(
            research=research_text,
            marketing=marketing_text,
            legal=legal_text,
        )

        # Provide a readable, consolidated string as the final workflow result.
        consolidated = (
            "Consolidated Insights\n"
            "====================\n\n"
            f"Research Findings:\n{aggregated.research}\n\n"
            f"Marketing Angle:\n{aggregated.marketing}\n\n"
            f"Legal/Compliance Notes:\n{aggregated.legal}\n"
        )

        await ctx.yield_output(consolidated)


print("✅ AggregateInsights executor defined")

## Create Domain Expert Agents

Configure three specialized AI agents using Azure OpenAI.

### Agent 1: Researcher
- **Role**: Expert market and product researcher
- **Instructions**: Provide concise, factual insights, opportunities, and risks
- **ID**: `"researcher"`
- **Focus**: Data-driven market analysis

### Agent 2: Marketer
- **Role**: Creative marketing strategist
- **Instructions**: Craft compelling value propositions and target messaging
- **ID**: `"marketer"`
- **Focus**: Customer-centric positioning and communication

### Agent 3: Legal
- **Role**: Cautious legal/compliance reviewer
- **Instructions**: Highlight constraints, disclaimers, and policy concerns
- **ID**: `"legal"`
- **Focus**: Risk mitigation and regulatory compliance

### AgentExecutor Wrapping:
- **Purpose**: Integrates Azure OpenAI agents into workflow
- **Chat Client**: Manages agent lifecycle and API calls
- **ID Assignment**: Enables targeted routing and response mapping

### Why Multiple Perspectives?
- **Comprehensive Analysis**: Cover all aspects of a business decision
- **Balanced View**: Temper optimism with realistic constraints
- **Stakeholder Alignment**: Address concerns of different departments
- **Risk Management**: Identify legal and compliance issues early

In [ ]:
# Create Azure OpenAI chat client
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)

# Create researcher agent
researcher = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You're an expert market and product researcher. Given a prompt, provide concise, factual insights,"
            " opportunities, and risks."
        ),
    ),
    id="researcher",
)
print("✅ Created researcher agent")
print("   Focus: Market analysis, opportunities, and risks")

# Create marketer agent
marketer = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You're a creative marketing strategist. Craft compelling value propositions and target messaging"
            " aligned to the prompt."
        ),
    ),
    id="marketer",
)
print("✅ Created marketer agent")
print("   Focus: Value propositions and target messaging")

# Create legal agent
legal = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, and policy concerns"
            " based on the prompt."
        ),
    ),
    id="legal",
)
print("✅ Created legal agent")
print("   Focus: Compliance, disclaimers, and policy concerns")

## Create Dispatcher and Aggregator

Instantiate the custom executors with the expert agent IDs.

In [ ]:
# Collect expert IDs
expert_ids = [researcher.id, marketer.id, legal.id]

# Create dispatcher
dispatcher = DispatchToExperts(expert_ids=expert_ids, id="dispatcher")
print("✅ Created dispatcher executor")
print(f"   Will dispatch to: {expert_ids}")

# Create aggregator
aggregator = AggregateInsights(expert_ids=expert_ids, id="aggregator")
print("✅ Created aggregator executor")
print(f"   Will aggregate from: {expert_ids}")

## Build the Workflow

Construct the fan-out/fan-in workflow graph.

### Workflow Structure:

1. **Entry Point**: `dispatcher`
2. **Fan-Out Edges**: `dispatcher` → `[researcher, marketer, legal]`
   - All three agents receive the prompt simultaneously
   - Agents process independently and in parallel
3. **Fan-In Edges**: `[researcher, marketer, legal]` → `aggregator`
   - Aggregator waits for all agents to complete
   - Responses collected as list in order

### Graph Visualization:
```
        dispatcher
       /    |    \
 researcher marketer legal
       \    |    /
       aggregator
```

### WorkflowBuilder API:
- **`set_start_executor()`**: Define entry point
- **`add_fan_out_edges()`**: Create parallel execution paths
- **`add_fan_in_edges()`**: Collect results from parallel paths
- **`build()`**: Finalize and validate workflow graph

In [ ]:
# Build the workflow
workflow = (
    WorkflowBuilder()
    .set_start_executor(dispatcher)
    .add_fan_out_edges(dispatcher, [researcher, marketer, legal])
    .add_fan_in_edges([researcher, marketer, legal], aggregator)
    .build()
)

print("✅ Workflow built successfully!")
print("\nWorkflow structure:")
print("   dispatcher → [researcher, marketer, legal] → aggregator")

## Generate Workflow Visualization

Use **WorkflowViz** to create visual representations of the workflow graph.

### Visualization Options:

#### 1. Mermaid Diagram
- **Format**: Text-based flowchart syntax
- **Usage**: Embed in markdown, documentation, GitHub README
- **Rendering**: Supported by GitHub, GitLab, VS Code, and many tools
- **Method**: `viz.to_mermaid()`
- **Live Preview**: Copy output to https://mermaid.live

#### 2. DiGraph (NetworkX)
- **Format**: NetworkX directed graph object
- **Usage**: Programmatic graph analysis and algorithms
- **Capabilities**: Compute centrality, paths, cycles, etc.
- **Method**: `viz.to_digraph()`
- **Analysis**: Access nodes, edges, and graph properties

#### 3. SVG Export
- **Format**: Scalable Vector Graphics
- **Usage**: High-quality documentation images
- **Requirements**: `pip install agent-framework[viz] --pre` + GraphViz binaries
- **Method**: `viz.export(format="svg")`
- **Output**: File path to generated SVG

### WorkflowViz API:
```python
viz = WorkflowViz(workflow)
mermaid_code = viz.to_mermaid()      # Get Mermaid diagram code
digraph = viz.to_digraph()           # Get NetworkX DiGraph
svg_file = viz.export(format="svg")  # Export as SVG (requires GraphViz)
```

In [ ]:
print("\n" + "="*60)
print("WORKFLOW VISUALIZATION")
print("="*60)

# Create WorkflowViz instance
viz = WorkflowViz(workflow)

# 1. Generate Mermaid diagram
mermaid_code = viz.to_mermaid()
print("\n📊 MERMAID DIAGRAM")
print("="*60)
print(mermaid_code)
print("="*60)
print("💡 Copy the above code to https://mermaid.live to visualize\n")

# 2. Generate DiGraph
digraph = viz.to_digraph()
print("\n🔗 DIGRAPH (NetworkX)")
print("="*60)
print(digraph)
print("="*60)
print(f"\n📊 Graph Statistics:")
print(f"   - Nodes: {digraph.number_of_nodes()}")
print(f"   - Edges: {digraph.number_of_edges()}")
print(f"   - Node list: {list(digraph.nodes())}")
print(f"   - Edge list: {list(digraph.edges())}")

# 3. Try to export as SVG
print("\n\n📄 SVG EXPORT")
print("="*60)
try:
    svg_file = viz.export(format="svg")
    print(f"✅ SVG file saved to: {svg_file}")
    print(f"   Open this file in a browser or image viewer to see the graph")
except ImportError as e:
    print("⚠️  SVG export requires additional dependencies")
    print("   Install with: pip install agent-framework[viz] --pre")
    print("   Also install GraphViz binaries: https://graphviz.org/download/")
except Exception as e:
    print(f"⚠️  SVG export failed: {e}")
    print("   Make sure GraphViz is installed and in your PATH")

print("\n" + "="*60)

## Define the Product Launch Prompt

Create a sample prompt to analyze from multiple expert perspectives.

### Sample Prompt:
```
We are launching a new budget-friendly electric bike for urban commuters.
```

### Expected Analysis:
- **Researcher**: Market size, competition, consumer trends, opportunities
- **Marketer**: Target audience, value proposition, messaging, channels
- **Legal**: Safety regulations, liability, warranties, environmental claims

### Try Your Own Prompts:
- "Developing an AI-powered personal finance app for Gen Z"
- "Launching a sustainable fashion brand with carbon-neutral shipping"
- "Creating a B2B SaaS platform for remote team collaboration"
- "Introducing a plant-based protein supplement for athletes"

In [ ]:
# Define the prompt
prompt = "We are launching a new budget-friendly electric bike for urban commuters."

print("\n📋 Product Launch Prompt:")
print(f"   {prompt}")
print("\n💡 This prompt will be analyzed by three expert agents:")
print("   🔬 Researcher: Market analysis and opportunities")
print("   📈 Marketer: Value propositions and messaging")
print("   ⚖️ Legal: Compliance and risk assessment")

## Run the Workflow with Event Streaming

Execute the workflow and monitor agent execution in real-time.

### Event Streaming:
- **`workflow.run_stream()`**: Returns async iterator of events
- **`AgentRunEvent`**: Emitted when agents start/complete processing
- **`WorkflowOutputEvent`**: Emitted when workflow yields final output
- **Real-Time Monitoring**: See execution progress as it happens

### Expected Flow:
1. **Dispatcher**: Receives prompt and fans out to all agents
2. **Parallel Processing**: All three agents process simultaneously
   - `AgentRunEvent` for researcher
   - `AgentRunEvent` for marketer
   - `AgentRunEvent` for legal
3. **Aggregator**: Collects and consolidates responses
4. **Output**: `WorkflowOutputEvent` with final consolidated report

### Event Types:
- **`AgentRunEvent`**: Contains agent ID, status, and response
- **`WorkflowOutputEvent`**: Contains final output data
- **Other Events**: Executor completion, message passing, etc.

### Note on Execution Order:
- Agents may complete in any order (parallel execution)
- Network latency affects completion timing
- Framework ensures all agents finish before aggregation

In [ ]:
print("\n" + "="*60)
print("RUNNING WORKFLOW")
print("="*60)

# Run the workflow with event streaming
async for event in workflow.run_stream(prompt):
    if isinstance(event, AgentRunEvent):
        # Show which agent ran and what step completed
        print(f"\n🤖 AgentRunEvent:")
        print(f"   Agent: {event.agent_id if hasattr(event, 'agent_id') else 'Unknown'}")
        print(f"   Event: {event}")
    elif isinstance(event, WorkflowOutputEvent):
        print("\n" + "="*60)
        print("📊 FINAL AGGREGATED OUTPUT")
        print("="*60)
        print(event.data)
        print("="*60)

print("\n✅ Workflow execution complete!")

## Key Takeaways

### Fan-Out/Fan-In Pattern
- ✅ **Parallel Processing**: Multiple agents work simultaneously for efficiency
- ✅ **Same Input, Multiple Perspectives**: Each agent analyzes from their expertise
- ✅ **Automatic Coordination**: Framework manages parallel execution and collection
- ✅ **Scalable Pattern**: Easy to add more domain experts

### Custom Executors with @handler
- ✅ **Flexible Logic**: Implement custom business logic in executors
- ✅ **@handler Decorator**: Registers method as executor's main processing function
- ✅ **Constructor Patterns**: Pass configuration (expert_ids) to executors
- ✅ **Type Safety**: Strong typing for inputs and outputs

### Multi-Agent Orchestration
- ✅ **Domain Expertise**: Each agent specializes in specific knowledge area
- ✅ **Target Routing**: `target_id` directs messages to specific agents
- ✅ **Response Mapping**: Organize outputs by `executor_id`
- ✅ **Structured Aggregation**: Dataclass for organized multi-agent insights

### Workflow Visualization
- ✅ **Mermaid Diagrams**: Text-based flowcharts for documentation
- ✅ **DiGraph Analysis**: NetworkX graphs for programmatic analysis
- ✅ **SVG Export**: High-quality vector graphics (requires GraphViz)
- ✅ **Documentation**: Visual aids improve workflow understanding

### Event Streaming
- ✅ **Real-Time Monitoring**: Track execution progress as it happens
- ✅ **AgentRunEvent**: Observe agent start/completion events
- ✅ **WorkflowOutputEvent**: Capture final results
- ✅ **Debugging Support**: Events provide detailed execution traces

### Azure OpenAI Integration
- ✅ **Enterprise Authentication**: AzureCliCredential for secure access
- ✅ **System Instructions**: Define agent behavior and expertise
- ✅ **AgentExecutor Wrapper**: Seamless workflow integration
- ✅ **Scalable Architecture**: Cloud-based LLM inference

### When to Use This Pattern
- ✅ Need multiple expert perspectives on same problem
- ✅ Complex analysis requiring diverse knowledge domains
- ✅ Want to parallelize AI processing for speed
- ✅ Building decision support systems
- ✅ Creating comprehensive reports from multiple viewpoints
- ✅ Need to visualize complex workflows

### Best Practices
- 🎯 **Define Clear Roles**: Give each agent specific expertise and focus
- 🎯 **Use Structured Output**: Dataclasses for organized aggregation
- 🎯 **Leverage Visualization**: Generate diagrams for documentation
- 🎯 **Monitor Execution**: Use event streaming for observability
- 🎯 **Handle Failures**: Implement error recovery for agent failures
- 🎯 **Document Workflows**: Include Mermaid diagrams in README files
- 🎯 **Test Parallelism**: Verify agents work independently

### Visualization Best Practices
- 🎯 **Mermaid for Docs**: Embed text-based diagrams in markdown
- 🎯 **SVG for Reports**: Use vector graphics in presentations
- 🎯 **DiGraph for Analysis**: Compute graph metrics programmatically
- 🎯 **Version Control**: Text-based Mermaid works with git diff
- 🎯 **Live Preview**: Use mermaid.live for quick visualization

### Performance Optimization
- 🚀 Use fan-out for embarrassingly parallel tasks
- 🚀 Minimize agent instruction length for faster responses
- 🚀 Cache agent responses when appropriate
- 🚀 Monitor event stream for bottlenecks
- 🚀 Consider timeout strategies for slow agents

### Next Steps
- Add more domain experts (financial, technical, ethical, etc.)
- Implement weighted aggregation (prioritize certain experts)
- Add consensus detection (identify agreement/disagreement)
- Create interactive visualization dashboard
- Integrate with project management tools
- Build web interface for multi-agent consultation
- Implement retry logic for failed agents
- Add A/B testing for different agent instructions
- Create templates for common analysis patterns